In [1]:
# necessary packages #

#using Pkg
#Pkg.add("Distances")
using Distributions
using Random
using Distances
using LinearAlgebra
using SparseArrays
using JLD2

In [2]:
include("../../../../util2.j")

colnorm (generic function with 1 method)

In [3]:
@load "../../data/sim3_pars.jld";

In [8]:
# Generate simulation data #
Random.seed!(1);
X = [fill(1, (N,)) rand(N) rand(N)];                             # design matrix
Dist = pairwise(Euclidean(), coords, dims = 1);                  # distance matrix
F_true = hcat([rand(MvNormal(exp.(-ϕ[i] * Dist)), 1) for i in 1:K]...);
ω = F_true * Λ;                                               # latent process
Y = X * β + ω + [rand(Normal(0.0, sqrt_Σ_diag[i]), 1)[1] for j in 1:N, i in 1:q];              # response matrix
ω_incp = ω + fill(1.0, (N, 1)) * transpose(β[1, :]);
X_ord = X[ordx, :]; Y_ord = Y[ordx, :]; ω_ord = ω[ordx, :];   # sorted data
ω_incp_obs = ω_ord + fill(1.0, (N, 1)) * transpose(β[1, :]); # latent process + intercept

In [9]:
@save "sim3data.jld"

In [8]:
@load "sim3data.jld"

57-element Array{Symbol,1}:
 Symbol("##1#2")  
 Symbol("##11#12")
 Symbol("##13#14")
 Symbol("##15#16")
 Symbol("##17#18")
 Symbol("##19#20")
 Symbol("##19#21")
 Symbol("##20#22")
 Symbol("##21#22")
 Symbol("##3#5")  
 Symbol("##4#6")  
 Symbol("##7#8")  
 Symbol("##9#10") 
 ⋮                
 :obs_ind         
 :ordx            
 :perm_ind        
 :q               
 :sqrt_Σ_diag     
 :Λ               
 :β               
 :ω               
 :ω_incp          
 :ω_incp_obs      
 :ω_ord           
 :ϕ               

In [9]:
using RCall
@rput ω_incp
@rput coords
R"""
library(MBA)
library(classInt)
library(RColorBrewer)
library(sp)
library(coda)
library(spBayes)
library(fields)
library(corrgram)

h <- 12
surf.w_incp1 <- mba.surf(cbind(coords, ω_incp[, 1]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp2 <- mba.surf(cbind(coords, ω_incp[, 2]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp3 <- mba.surf(cbind(coords, ω_incp[, 3]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp4 <- mba.surf(cbind(coords, ω_incp[, 4]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp5 <- mba.surf(cbind(coords, ω_incp[, 5]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp6 <- mba.surf(cbind(coords, ω_incp[, 6]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp7 <- mba.surf(cbind(coords, ω_incp[, 7]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp8 <- mba.surf(cbind(coords, ω_incp[, 8]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp9 <- mba.surf(cbind(coords, ω_incp[, 9]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est
surf.w_incp10 <- mba.surf(cbind(coords, ω_incp[, 10]), no.X = 300, no.Y = 300, 
              exten = TRUE, sp = TRUE, h = h)$xyz.est

surf.brks <- classIntervals(c(surf.w_incp1[["z"]], surf.w_incp2[["z"]], surf.w_incp3[["z"]], surf.w_incp4[["z"]],
                surf.w_incp5[["z"]], surf.w_incp6[["z"]], surf.w_incp7[["z"]], surf.w_incp8[["z"]],
                surf.w_incp9[["z"]], surf.w_incp10[["z"]]),500, 'pretty')$brks
col.pal <- colorRampPalette(brewer.pal(11,'RdBu')[11:1])
xlim <- c(0, 1.13)

zlim <- range(c(surf.w_incp1[["z"]], surf.w_incp2[["z"]], surf.w_incp3[["z"]], surf.w_incp4[["z"]],
                surf.w_incp5[["z"]], surf.w_incp6[["z"]], surf.w_incp7[["z"]], surf.w_incp8[["z"]],
                surf.w_incp9[["z"]], surf.w_incp10[["z"]]))

# size for the mapping of w               
width <- 720
height <- 720
pointsize <- 16

png(paste("../pics/sim3_map-w1_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp1)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w2_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp2)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w3_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp3)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w4_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp4)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w5_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp5)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w6_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp6)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w7_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp7)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w8_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp8)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w9_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp9)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()

png(paste("../pics/sim3_map-w10_incp-true.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
##Obs
i <- as.image.SpatialGridDataFrame(surf.w_incp10)
plot(coords, typ="n", cex=0.5, xlim=xlim, axes=FALSE, ylab="y", xlab="x") 
     #main = "true")
axis(2, las=1)
axis(1)
image.plot(i, add=TRUE, col=rev(col.pal(length(surf.brks)-1)), zlim=zlim)
dev.off()


# correlation maps #
corr_resp = cor(ω_incp, use = "pairwise.complete.obs");
par(mfrow = c(1,1))
colnames(corr_resp) <- c(1:dim(ω_incp)[2])
rownames(corr_resp) <- c(1:dim(ω_incp)[2])
png(paste("../pics/sim3_factor_corr_plot_raw.png", sep = ""), 
    width = width, height = height, pointsize = pointsize, family = "Courier")
par(mfrow = c(1, 1))
corrgram(corr_resp, order=FALSE, lower.panel=panel.shade, gap = 0.2,
         upper.panel=panel.pie, text.panel=panel.txt, main=" ",
         col.regions = colorRampPalette(c( "darkseagreen3",
                                           "white", "cadetblue3")))
dev.off()

"""

┌ Warning: RCall.jl: Loading required package: magic
│ Loading required package: abind
│ Loading required package: Formula
│ Loading required package: Matrix
└ @ RCall /home/lu/.julia/packages/RCall/g7dhB/src/io.jl:113
┌ Warning: RCall.jl: Loading required package: spam
│ Loading required package: dotCall64
│ Loading required package: grid
│ Spam version 2.2-2 (2019-03-07) is loaded.
│ Type 'help( Spam)' or 'demo( spam)' for a short introduction 
│ and overview of this package.
│ Help for individual functions is also obtained by adding the
│ suffix '.spam' to the function name, e.g. 'help( chol.spam)'.
│ 
│ Attaching package: ‘spam’
│ 
│ The following object is masked from ‘package:Matrix’:
│ 
│     det
│ 
│ The following objects are masked from ‘package:base’:
│ 
│     backsolve, forwardsolve
│ 
│ Loading required package: maps
│ See https://github.com/NCAR/Fields for
│  an extensive vignette, other supplements and source code 
└ @ RCall /home/lu/.julia/packages/RCall/g7dhB/src/io.jl:

RObject{IntSxp}
png 
  2 
